In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, ElectraTokenizer, BertForSequenceClassification, ElectraForSequenceClassification
from tqdm.notebook import tqdm
import re
import torch.nn.functional as F
import numpy as np


In [2]:

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters and numbers
    text = re.sub(r'[^가-힣0-9]', ' ', text)
    
    # Remove extra spaces
    text = ' '.join(text.split())
    
    # 한국어 불용어 리스트 - 년, 또, 그런, 좀, 잘, 개, 아니, 씨, 안, 다시, 못하
    stopwords = [
        '이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', 
        '등', '같', '우리', '때', '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하', 
        '때문', '그것', '두', '말하', '알', '그러나', '받', '일', '문제', '더', '사회', 
        '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '시키', '만들', '지금', '생각하', 
        '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신', '어떤', '내', '경우',
        '명', '생각', '시간', '그녀', '이런', '앞', '보이', '번', '나', '다른', '어떻', '여자',
        '전', '들', '사실', '이렇', '점', '싶', '말', '정도', '원', '통하', '소리', '놓'
    ]
    
    # 불용어 제거
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    return text

In [3]:
# Define the conversation column names for each file based on the earlier inspection
conversation_columns = {
    "./data/train_sr_cleaned.csv": "conversation_sr_cleaned",
    "./data/train_augmented_wv_.csv": "conversation",
    "./data/LLaMa2_Augmentation_trian.csv": "conversation",
    "./data/train.csv": "conversation"
}

# Load and preprocess the data from each file, then concatenate them
all_dataframes = []

for file_path, conv_column in conversation_columns.items():
    df_temp = pd.read_csv(file_path)
    df_temp = df_temp[['class', conv_column]]
    df_temp.columns = ['class', 'conversation']  # Renaming columns for uniformity
    df_temp['conversation'] = df_temp['conversation'].apply(clean_text)
    all_dataframes.append(df_temp)

# Concatenate all the dataframes
merged_data = pd.concat(all_dataframes, ignore_index=True)

test = pd.read_json('./data/test.json').transpose()

# train 데이터의 텍스트 열 정규화
merged_data['conversation'] = merged_data['conversation'].apply(clean_text)
test['conversation'] = test['text'].apply(clean_text)

# 지정된 클래스를 숫자로 인코딩
label_dict = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3
}
merged_data['label_encoded'] = merged_data['class'].map(label_dict)

merged_data.drop_duplicates(subset=['conversation'], inplace=True)


# NaN 값을 가진 행 제거
merged_data.dropna(subset=['conversation', 'label_encoded'], inplace=True)

# 빈 문자열 값을 가진 행 제거
merged_data = merged_data[merged_data['conversation'] != ""]

merged_data.reset_index(drop=True, inplace=True)


In [4]:
print(merged_data.head())


       class                                       conversation  label_encoded
0      협박 대화  너 스스로를 죽여달라고 애원하는 인가 아닙니다 죄송합니다 죽을 거면 혼자 죽지 사건...              0
1      협박 대화  길동판단입니다꽤 시 분 마트에 폭발물을 설치할거다 죽는거야 똑바로 들어 한번만 얘기...              0
2  기타 괴롭힘 대화  너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지...              3
3      갈취 대화  어이 거기 예 너 말이야 너 동안 이리 오라고 사이 무슨 너 늑대 옷 좋아보인다 얘...              1
4      갈취 대화  저기요 혹시 한데이 너무 뜨겁잖아요 저희 회사에서 알아보던 선크림 파는데 손등에 발...              1


In [5]:

# 토크나이저 로드
klue_tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# KLUE용 Dataset 정의
class KLUEDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['conversation' if 'conversation' in dataframe else 'text'].tolist()
        self.labels = dataframe['label_encoded'].tolist() if 'label_encoded' in dataframe else [0] * len(dataframe)
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        try:
            text = str(self.text[index])
            inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                max_length=self.max_len,
                pad_to_max_length=True,
                return_token_type_ids=True
            )
            ids = inputs['input_ids']
            mask = inputs['attention_mask']

            return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.labels[index], dtype=torch.long)
            }
        except Exception as e:
            print(f"Error at index {index}: {e}")
            print(f"Text: {self.text[index]}, Label: {self.labels[index]}")
            raise

class KLUEClass(torch.nn.Module):
    def __init__(self):
        super(KLUEClass, self).__init__()
        self.l1 = BertForSequenceClassification.from_pretrained('klue/bert-base', num_labels=4)
        
    def forward(self, ids, mask):
        output = self.l1(ids, attention_mask=mask, return_dict=True)
        return output.logits

def predict(model, data_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for _, data in enumerate(data_loader):
            ids = data['ids'].to(torch.device("cuda"), dtype=torch.long)
            mask = data['mask'].to(torch.device("cuda"), dtype=torch.long)
            
            outputs = model(ids, mask)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    return predictions

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [ ]:
from tqdm import tqdm

results = []

# 토큰 최대 길이를 128에서 480까지 16씩 증가시키며 조정
for MAX_LEN in tqdm(range(128, 481, 16), desc="Overall Progress"):
    klue_train_dataset = KLUEDataset(merged_data, klue_tokenizer, MAX_LEN)
    klue_train_data_loader = DataLoader(klue_train_dataset, batch_size=16)
    
    klue_model = KLUEClass()
    klue_model.to(torch.device("cuda"))
    
    loss_function = torch.nn.CrossEntropyLoss()
    klue_optimizer = torch.optim.Adam(params=klue_model.parameters(), lr=1e-5)
    
    for epoch in tqdm(range(2), desc=f"Training for MAX_LEN={MAX_LEN}", leave=False):
        klue_model.train()
        for _, data in enumerate(klue_train_data_loader, start=1):
            ids = data['ids'].to(torch.device("cuda"), dtype=torch.long)
            mask = data['mask'].to(torch.device("cuda"), dtype=torch.long)
            targets = data['targets'].to(torch.device("cuda"), dtype=torch.long)
            
            outputs = klue_model(ids, mask)
            loss = loss_function(outputs, targets)
            
            klue_optimizer.zero_grad()
            loss.backward()
            klue_optimizer.step()
    
    # test.json을 사용하여 모델 평가하고 submission.csv에 결과 저장
    test_dataset = KLUEDataset(test, klue_tokenizer, MAX_LEN)
    test_data_loader = DataLoader(test_dataset, batch_size=16)

    test_predictions = predict(klue_model, test_data_loader)

    submission_df = pd.DataFrame(test_predictions, columns=['class'])
    submission_df.to_csv(f"./data/klue_3rd_trial_different_stopwards/submission_{MAX_LEN}.csv", index=False)
    
    # submission.csv와 answer.csv를 비교하여 정확도 계산
    answer_df = pd.read_csv("./data/answer.csv")
    correct_predictions = (submission_df['class'] == answer_df['class']).sum()
    accuracy = correct_predictions / len(answer_df)
    print(f'accuracy = {accuracy}')
    
    results.append(accuracy)
    
    if accuracy >= 0.91:
        model_name = f"klue_{accuracy:.4f}_{MAX_LEN}"
        torch.save(klue_model.state_dict(), f"./data/klue_3rd_trial_different_stopwards/{model_name}.pth")

results


Overall Progress:   0%|          | 0/23 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

accuracy = 0.88


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

...

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

accuracy = 0.92


Overall Progress:  57%|█████▋    | 13/23 [5:00:41<4:50:56, 1745.66s/it]Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

accuracy = 0.9


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [ ]:
import matplotlib.pyplot as plt

# Define the range of MAX_LEN values
MAX_LEN_values = list(range(128, 481, 16))

# Plot the accuracy values for each MAX_LEN
plt.figure(figsize=(12, 6))
plt.plot(MAX_LEN_values, results, marker='o')
plt.xlabel('MAX_LEN')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. MAX_LEN')
plt.grid(True)
plt.show()
